In [1]:
from torchvision.datasets import ImageFolder
from tqdm import tqdm

In [ ]:
reference_images = ImageFolder(root='../../datasets/armbench/Reference_Images/',)

In [ ]:
pick_images= ImageFolder(root='../../datasets/armbench/Picks',)

In [8]:
import pandas as pd
obj = pd.read_pickle('../../datasets/armbench/armbench-object-id/train-test-split.pickle')

In [3]:
obj

{'trainset': ['028023001665774539482',
  '020023001665774539482',
  '017023001665774539482',
  '016023001665774539482',
  '012023001665774539482',
  '010023001665774539482',
  '002023001665774539482',
  '011022001665774539482',
  '010022001665774539482',
  '003022001665774539482',
  '006021001665774539482',
  '004021001665774539482',
  '002021001665774539482',
  '001021001665774539482',
  '005020001665774539482',
  '004020001665774539482',
  '003020001665774539482',
  '002020001665774539482',
  '001020001665774539482',
  '018019001665774539482',
  '010019001665774539482',
  '006019001665774539482',
  '004019001665774539482',
  '003019001665774539482',
  '010018001665774539482',
  '009018001665774539482',
  '008018001665774539482',
  '007018001665774539482',
  '006018001665774539482',
  '003018001665774539482',
  '004015001665774539482',
  '002015001665774539482',
  '003013001665774539482',
  '002013001665774539482',
  '004012001665774539482',
  '003012001665774539482',
  '0010120016657

In [7]:
import json
with open('../../datasets/armbench/armbench-object-id\Picks/001001001651711238293/container.json', 'r') as f:
    annotation = json.load(f)

In [13]:
obj.keys()

dict_keys(['trainset', 'testset', 'trainset-objects', 'testset-objects'])

In [11]:
annotation.keys()

dict_keys(['B00Q70RCW6', 'X002UQYD3D', 'X002SQZ6NB', 'X0029URFHV'])

In [ ]:
import json
import os
import numpy as np
from tqdm import tqdm
label_dict = {}
for file in tqdm(os.listdir('../../datasets/armbench/armbench-object-id/Picks/')):
    with open(f'../../datasets/armbench/armbench-object-id/Picks/{file}/annotation.json', 'r') as f:
        annotation = json.load(f)
    label_dict[file] = annotation["GT_ID"]

# np.save('../../datasets/armbench/armbench-object-id/all_pickid_to_itemid.npy', label_dict)

In [20]:
with open('../../datasets/armbench/armbench-object-id/all_labels.json', 'r') as f:
    label_dict = json.load(f)

In [ ]:
label_dict

In [27]:
train_pickids = obj['testset']

In [31]:
from tqdm import tqdm
import os
import numpy as np
import shutil
pick_up_classes = train_pickids.copy()
training_classes_3t1 = []
training_data = {}
all_training_data = {}
ref_img_paths = {}
pick_img_paths = {}
train_data_one_to_one = {}

for clsidx in tqdm(range(len(train_pickids))):
    all_img_paths = []
    image_paths = []
    ref_paths = []
    for path in os.listdir(f'../../datasets/armbench/armbench-object-id/Picks/{train_pickids[clsidx]}/'):
        if path.endswith('.jpg'):
            all_img_paths.append(path)
            if 'FarTrayRGB' not in path:
                image_paths.append(path)
    
    with open(f'../../datasets/armbench/armbench-object-id/Picks/{train_pickids[clsidx]}/container.json', 'r') as f:
        annotation = json.load(f)
    potential_ref_id = list(annotation.keys())
    
    ref_gallary_paths = []
    ref_gallary_ids = []
    for id in potential_ref_id:
        try:
            for path in os.listdir(f'../../datasets/armbench/armbench-object-id/Reference_Images/{id}/'):
                if path.endswith('.jpg'):
                    ref_gallary_paths.append(os.path.join(id , path))
                    ref_gallary_ids.append(id)
        except:
            continue
    
    ref_id = label_dict[train_pickids[clsidx]]
    try:
        for path in os.listdir(f'../../datasets/armbench/armbench-object-id/Reference_Images/{ref_id}/'):
            if path.endswith('.jpg'):
                ref_paths.append(path)

        pick_img_paths[train_pickids[clsidx]] = all_img_paths
        ref_img_paths[ref_id] = ref_paths
        if len(ref_paths)==0:
            continue

        # for path in ref_paths:
        #     train_data_one_to_one[train_pickids[clsidx]] = 

        if len(image_paths) < 3:
            all_training_data[train_pickids[clsidx]] = {'pickimg_paths': all_img_paths,
                                                'ref_id': ref_id,
                                                'refimg_path': ref_paths,
                                                        'ref_gallary_paths': ref_gallary_paths, 'ref_gallary_ids': ref_gallary_ids,
                                                        
                                                        }
            continue
        assert len(image_paths) == 3
        training_classes_3t1.append(train_pickids[clsidx])
        training_data[train_pickids[clsidx]] = {'pickimg_paths': all_img_paths,
                                                'pickimg_3t1_paths': image_paths,
                                                'ref_id': ref_id,
                                                'refimg_path': ref_paths,
                                                'ref_gallary_paths': ref_gallary_paths, 'ref_gallary_ids': ref_gallary_ids}
    except:
        print('No ref image found for pick id: ', train_pickids[clsidx])
        
    # try:
    #     for path in os.listdir(f'../../datasets/armbench/armbench-object-id/Reference_Images/{ref_id}/'):
    #         if path.endswith('.jpg'):
    #             ref_paths.append(path)
    # 
    #     pick_img_paths[train_pickids[clsidx]] = all_img_paths
    #     ref_img_paths[ref_id] = ref_paths
    #     if len(ref_paths)==0:
    #         continue
    # 
    #     # for path in ref_paths:
    #     #     train_data_one_to_one[train_pickids[clsidx]] = 
    # 
    #     if len(image_paths) < 3:
    #         all_training_data[train_pickids[clsidx]] = {'pickimg_paths': all_img_paths,
    #                                             'ref_id': ref_id,
    #                                             'refimg_path': ref_paths}
    #         continue
    #     assert len(image_paths) == 3
    #     training_classes_3t1.append(train_pickids[clsidx])
    #     training_data[train_pickids[clsidx]] = {'pickimg_paths': all_img_paths,
    #                                             'pickimg_3t1_paths': image_paths,
    #                                             'ref_id': ref_id,
    #                                             'refimg_path': ref_paths}
    # except:
    #     continue

# np.save('../../datasets/armbench/armbench-object-id/train_data_over3image.npy', training_data)
with open('../../datasets/armbench/armbench-object-id/test_data_3t1.json', 'w') as f:
    json.dump(training_data, f)
# # np.save('../../datasets/armbench/armbench-object-id/all_training.npy', all_training_data)
with open('../../datasets/armbench/armbench-object-id/all_test_data.json', 'w') as f:
    json.dump(all_training_data, f)
# # np.save('../../datasets/armbench/armbench-object-id/all_train_pick_img_paths.npy', pick_img_paths)
# with open('../../datasets/armbench/armbench-object-id/all_train_pick_img_paths.json', 'w') as f:
#     json.dump(pick_img_paths, f)
# # np.save('../../datasets/armbench/armbench-object-id/all_train_ref_img_paths.npy', ref_img_paths)
# with open('../../datasets/armbench/armbench-object-id/all_train_ref_img_paths.json', 'w') as f:
#     json.dump(ref_img_paths, f)

100%|██████████| 50000/50000 [00:53<00:00, 936.34it/s] 


In [25]:
import json
with open('../../datasets/armbench/armbench-object-id/train_data_3t1.json', 'r') as f:
    data = json.load(f)

In [26]:
data.keys()

dict_keys(['028023001665774539482', '020023001665774539482', '016023001665774539482', '012023001665774539482', '010023001665774539482', '002023001665774539482', '011022001665774539482', '010022001665774539482', '003022001665774539482', '006021001665774539482', '004021001665774539482', '002021001665774539482', '001021001665774539482', '005020001665774539482', '004020001665774539482', '003020001665774539482', '001020001665774539482', '018019001665774539482', '010019001665774539482', '006019001665774539482', '004019001665774539482', '003019001665774539482', '010018001665774539482', '009018001665774539482', '008018001665774539482', '007018001665774539482', '003018001665774539482', '004015001665774539482', '002015001665774539482', '003013001665774539482', '002013001665774539482', '004012001665774539482', '003012001665774539482', '001012001665774539482', '006011001665774539482', '005011001665774539482', '007010001665774539482', '006010001665774539482', '011009001665774539482', '0090090016657

In [27]:
data['002001001651635353046']

{'pickimg_paths': ['FarTrayRGB.jpg',
  'OnArmLowRGB.jpg',
  'PickRGB.jpg',
  'ToteWallRGB.jpg'],
 'pickimg_3t1_paths': ['OnArmLowRGB.jpg', 'PickRGB.jpg', 'ToteWallRGB.jpg'],
 'ref_id': 'X0030K12QJ',
 'refimg_path': ['1.jpg', '2.jpg', '3.jpg', '4.jpg']}

In [32]:
with open('../../datasets/armbench/armbench-object-id/all_labels.json', 'w') as f:
    json.dump(label_dict, f)

In [ ]:
 path = os.path.join(self.data_path, 'Reference_Images', ref_id, image_paths[0])

In [33]:
i = 0
data_path = '../../datasets/armbench/armbench-object-id/'
for id in os.listdir(os.path.join(data_path, 'Reference_Images')):
    for file in os.listdir(os.path.join(data_path, 'Reference_Images', id)):
        if file.endswith(".jpg"):
            i += 1
i

749726

In [34]:
a = [434,234,5435,123,4324]
b = [2,4]
a[b]

TypeError: list indices must be integers or slices, not list